In [1]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install Flask flask-cors pyngrok


In [6]:
!ngrok authtoken 2ic5RCxXYzFWOjuIlPwBVgH20Dm_2Dz5YiKXsUoG5AKqwwSyD
#yourToken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [12]:
from flask import Flask, request, jsonify
import tensorflow.keras as keras
import numpy as np
from PIL import Image
import io
import cv2
from tensorflow.keras.preprocessing.image import array_to_img
from flask_cors import CORS
from pyngrok import ngrok
import threading

app = Flask(__name__)
CORS(app)  # CORS'u etkinleştir

# Load the model
model = keras.models.load_model('/content/drive/MyDrive/elif/model/model_ckpt.h5')

# Define class labels
class_indices = {
    0: 'Chronic apical periodontitis with vertical bone loss',
    1: 'Dental caries (proximal)',
    2: 'Embeded tooth',
    3: 'Impacted tooth (fully bony impaction)',
    4: 'Irreversible pulpitis with Acute periodontitis',
    5: 'Periodontitis',
    6: 'Improper restoration with chronic apical periodontitis'
}

def preprocess_image(image):
    if image is None:
        return None

    # Apply Gaussian blur to the image
    image = cv2.GaussianBlur(np.array(image), (5, 5), 0)
    image = array_to_img(image)
    # Expand dimensions to match model input shape
    image = np.expand_dims(image, axis=0)

    return image

def predict_image(image):
    image = preprocess_image(image)
    prediction = model.predict(image)
    # Get the predicted class index
    predicted_class_index = np.argmax(prediction)
    # Get the predicted class label using class_indices dictionary
    predicted_class = class_indices[predicted_class_index]
    return predicted_class

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    image = Image.open(io.BytesIO(file.read()))
    prediction = predict_image(image)

    return jsonify({'prediction': prediction})

def run_flask():
    # Flask uygulamasını 7000 portunda çalıştır
    app.run(host='0.0.0.0', port=7000)

def start_ngrok():
    # Ngrok'u 6000 portu için başlat
    public_url = ngrok.connect(7000)
    print(f"Ngrok Tunnel URL: {public_url}")

if __name__ == '__main__':
    # Flask uygulamasını ayrı bir iş parçacığında çalıştır
    flask_thread = threading.Thread(target=run_flask)
    flask_thread.start()

    # Ngrok'u başlat
    start_ngrok()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:7000
 * Running on http://172.28.0.12:7000
INFO:werkzeug:Press CTRL+C to quit


Ngrok Tunnel URL: NgrokTunnel: "https://e60d-34-83-101-143.ngrok-free.app" -> "http://localhost:7000"
